In [24]:
from scapy.all import rdpcap, UDP, IP
import datetime
import re
from tqdm import tqdm

pcap_file = "measurements/test2.pcap"
entry_file = "measurements/entries_2.txt"

packets = rdpcap(pcap_file)

entry_timestamps = []
packet_timestamps = []

for pkt in tqdm(packets):
    if IP in pkt and UDP in pkt:
        if pkt[IP].dst == "10.0.0.1":
            ts = pkt.time
            ts = float(pkt.time)
            sec = int(ts)
            nsec = int((ts - sec) * 1_000_000_000)

            formatted = f"{datetime.datetime.fromtimestamp(sec).strftime('%H:%M:%S')}.{nsec:09d}"
            packet_timestamps.append(formatted)

with open(entry_file) as f:
    lines = f.readlines()
    add_lines = []
    timestamps = []
    for line in tqdm(lines):
        if "Adding entry at:" in line:
            add_lines.append(line.strip())
    for line in tqdm(add_lines):
        match = re.search(r"Adding entry at:\s+(\d{2}:\d{2}:\d{2}\.\d+)", line)
        if match:
            timestamp = match.group(1)
            #print("Extracted timestamp:", timestamp)
            timestamps.append(timestamp)
        else:
            print("No timestamp found")
    #print(timestamps)
    entry_timestamps = timestamps


100%|██████████| 304/304 [00:00<00:00, 987659.50it/s]


In [ ]:
def to_ns_timestamp(tstr):
    hh, mm, rest = tstr.split(":")
    ss, ns = rest.split(".")
    hh = int(hh)
    mm = int(mm)
    ss = int(ss)
    ns = int(ns)
    total_ns = ((hh*3600 + mm*60 + ss) * 1_000_000_000) + ns
    return total_ns


entry_ns = [to_ns_timestamp(t) for t in entry_timestamps]
packet_ns = [to_ns_timestamp(t) for t in packet_timestamps]

packet_ns.sort()

latencies = []

for entry_time in entry_ns:
    first_packet_after_entry = None

    for packet_time in packet_ns:
        if packet_time > entry_time:
            first_packet_after_entry = packet_time
            break

    if first_packet_after_entry is None:
        latencies.append(None)
        continue

    latency = first_packet_after_entry - entry_time
    latencies.append(latency)

latencies_cleaned = [l for l in latencies if l is not None]
print(f"\nDurchschnittliche Latenz: {sum(latencies_cleaned)/len(latencies_cleaned)/1_000:.3f} ns")
print(f"Maximale Latenz: {max(latencies_cleaned)/1_000:.3f} ns")
print(f"Minimale Latenz: {min(latencies_cleaned)/1_000:.3f} ns")


Durchschnittliche Latenz: 2.243 ms
Maximale Latenz: 38.702 ms
Minimale Latenz: 0.100 ms
